In [ ]:
#fix import issues
import sys
from pathlib import Path
pkg_root = Path().resolve().parent
sys.path.insert(0, str(pkg_root))
__package__ = "chat_renderer"
from . import Chat

In [ ]:
chat = Chat.fromKey("1216633")
doc = chat.build()
with open("test.md", "w") as f:
    f.writelines(doc.render())

In [ ]:
editedFiles = list(filter(
            lambda path: len(chat.files[path]) > 1,
            chat.editedFiles
        ))

In [ ]:
editedFiles

In [ ]:
chat.files

In [ ]:
with open("foobar.json", "w") as f:
    from renderer import DB
    import json
    json.dump(DB.getDocument("chat-logs", "1096206"), f)

In [ ]:
class dummy:
    @staticmethod
    def foo():
        print("foo")

dummy.foo = lambda : print("bar")

In [ ]:
dummy.foo()

In [ ]:
from pathlib import PurePath

PurePath("/foo/bar") / "baz"

In [ ]:
set(1,2).size